In [1]:
import sys
import pandas as pd
import numpy as np

# Dodaj src do sys.path, by importować własne moduły
sys.path.append('../src')

from experiment_logger import log_experiment

In [2]:
TRAIN_PATH = '../../playground-series-s5e7/train.csv'
TEST_PATH = '../../playground-series-s5e7/test.csv'

train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)

print('Train shape:', train_data.shape)
print('Test shape:', test_data.shape)
train_data.head()

Train shape: (18524, 9)
Test shape: (6175, 8)


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [3]:
train_data.isna().sum()

id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64

In [4]:
test_data.isna().sum()

id                             0
Time_spent_Alone             425
Stage_fear                   598
Social_event_attendance      397
Going_outside                466
Drained_after_socializing    432
Friends_circle_size          350
Post_frequency               408
dtype: int64

In [5]:
train_data.info()
train_data.describe()
train_data['Personality'].value_counts(normalize=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 16631 non-null  object 
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  object 
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB


Personality
Extrovert    0.739527
Introvert    0.260473
Name: proportion, dtype: float64

In [6]:
train_data.drop(columns=['id'], inplace=True)
test_data.drop(columns=['id'], inplace=True)

In [7]:
# Załóżmy, że usunąłeś już kolumnę 'id' z train i test
from data_utils import split_numerical_categorical
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

# Zakładam, że X_train to Twój DataFrame z cechami (bez targetu i id)
target = train_data['Personality']
train_data.drop(columns=['Personality'], inplace=True)
numerical_cols, categorical_cols = split_numerical_categorical(train_data)

print("Zmienne numeryczne:", numerical_cols)
print("Zmienne kategoryczne:", categorical_cols)

# Łączymy train i test, by mieć spójny encoding i imputację
full = pd.concat([train_data, test_data], axis=0, ignore_index=True)

# Imputacja numeryczna
imputer = IterativeImputer(random_state=42)
full[numerical_cols] = imputer.fit_transform(full[numerical_cols])

# Imputacja kategoryczna - wypełnienie braków 'Missing'
for col in categorical_cols:
    full[col] = full[col].fillna('Missing')

print("Kolumny w full:", full.columns.tolist())
print("Kolumny kategorialne:", categorical_cols)

# One-hot encoding
full_encoded = pd.get_dummies(full, columns=categorical_cols)

# Rozdzielamy z powrotem
X_train = full_encoded.iloc[:len(train_data)]
X_test = full_encoded.iloc[len(train_data):]
y_train = target

Zmienne numeryczne: ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
Zmienne kategoryczne: ['Stage_fear', 'Drained_after_socializing']
Kolumny w full: ['Time_spent_Alone', 'Stage_fear', 'Social_event_attendance', 'Going_outside', 'Drained_after_socializing', 'Friends_circle_size', 'Post_frequency']
Kolumny kategorialne: ['Stage_fear', 'Drained_after_socializing']


In [8]:
X_train.head()

,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency,Stage_fear_Missing,Stage_fear_No,Stage_fear_Yes,Drained_after_socializing_Missing,Drained_after_socializing_No,Drained_after_socializing_Yes
0,0.0,6.0,4.0,15.0,5.000000,False,True,False,False,True,False
1,1.0,7.0,3.0,10.0,8.000000,False,True,False,False,True,False
2,6.0,1.0,0.0,3.0,0.000000,False,False,True,True,False,False
3,3.0,7.0,3.0,11.0,5.000000,False,True,False,False,True,False
4,1.0,4.0,4.0,13.0,5.708436,False,True,False,False,True,False


In [9]:
print(X_train.shape)
print(X_test.shape)

(18524, 11)
(6175, 11)


In [10]:
print(X_test.dtypes)
print(X_train.dtypes)
print(X_test.head())

Time_spent_Alone                     float64
Social_event_attendance              float64
Going_outside                        float64
Friends_circle_size                  float64
Post_frequency                       float64
Stage_fear_Missing                      bool
Stage_fear_No                           bool
Stage_fear_Yes                          bool
Drained_after_socializing_Missing       bool
Drained_after_socializing_No            bool
Drained_after_socializing_Yes           bool
dtype: object
Time_spent_Alone                     float64
Social_event_attendance              float64
Going_outside                        float64
Friends_circle_size                  float64
Post_frequency                       float64
Stage_fear_Missing                      bool
Stage_fear_No                           bool
Stage_fear_Yes                          bool
Drained_after_socializing_Missing       bool
Drained_after_socializing_No            bool
Drained_after_socializing_Yes           b

In [11]:
# Zakoduj target na liczby
y_train = y_train.map({'Extrovert': 0, 'Introvert': 1})

# Sprawdź, czy wszystko jest OK
print("Unikalne wartości y_train:", y_train.unique())
print("Typ y_train:", y_train.dtype)

Unikalne wartości y_train: [0 1]
Typ y_train: int64


In [12]:
# %pip install optuna
# 1. Importy
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import optuna
import lightgbm as lgb

# 2. Optuna - optymalizacja hiperparametrów
def objective_lgbm(trial):
    param_lgbm = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 15, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
        'random_state': 42,
        'n_jobs': -1
    }
    skf_lgbm = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores_lgbm = []
    for train_idx, val_idx in skf_lgbm.split(X_train, y_train):
        X_tr_lgbm, X_val_lgbm = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr_lgbm, y_val_lgbm = y_train.iloc[train_idx], y_train.iloc[val_idx]
        model_lgbm = lgb.LGBMClassifier(**param_lgbm)
        model_lgbm.fit(
            X_tr_lgbm, y_tr_lgbm,
            eval_set=[(X_val_lgbm, y_val_lgbm)],
            callbacks=[lgb.early_stopping(50, verbose=False)]
        )
        val_pred_lgbm = model_lgbm.predict(X_val_lgbm)
        score_lgbm = accuracy_score(y_val_lgbm, val_pred_lgbm)
        scores_lgbm.append(score_lgbm)
    return np.mean(scores_lgbm)

study_lgbm = optuna.create_study(direction='maximize')
study_lgbm.optimize(objective_lgbm, n_trials=30)  # Możesz zwiększyć n_trials dla lepszych wyników

print("Najlepsze parametry:", study_lgbm.best_params)

# 3. Finalny model z OOF predictions i predykcjami na testach
best_params_lgbm = study_lgbm.best_params
best_params_lgbm.update({'random_state': 42, 'n_jobs': -1})

skf_lgbm = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores_lgbm = []
test_preds_lgbm = np.zeros((len(X_test), skf_lgbm.n_splits))
oof_preds_lgbm = np.zeros(len(X_train))

for fold, (train_idx, val_idx) in enumerate(skf_lgbm.split(X_train, y_train)):
    X_tr_lgbm, X_val_lgbm = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr_lgbm, y_val_lgbm = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    model_lgbm = lgb.LGBMClassifier(**best_params_lgbm)
    model_lgbm.fit(
        X_tr_lgbm, y_tr_lgbm,
        eval_set=[(X_val_lgbm, y_val_lgbm)],
        callbacks=[lgb.early_stopping(50, verbose=False)]
    )
    
    val_pred_lgbm = model_lgbm.predict(X_val_lgbm)
    score_lgbm = accuracy_score(y_val_lgbm, val_pred_lgbm)
    scores_lgbm.append(score_lgbm)
    
    # OOF predictions
    oof_preds_lgbm[val_idx] = model_lgbm.predict_proba(X_val_lgbm)[:, 1]
    # Test predictions
    test_preds_lgbm[:, fold] = model_lgbm.predict_proba(X_test)[:, 1]

# OOF accuracy
oof_binary_lgbm = (oof_preds_lgbm > 0.5).astype(int)
print("OOF accuracy:", accuracy_score(y_train, oof_binary_lgbm))

# Uśrednienie predykcji po foldach
mean_preds_lgbm = test_preds_lgbm.mean(axis=1)
final_test_pred_lgbm = (mean_preds_lgbm > 0.5).astype(int)
print(f'Fold accuracy: {scores_lgbm}')
print(f'Mean CV accuracy: {np.mean(scores_lgbm):.4f}')

c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-07-02 13:43:04,712] A new study created in memory with name: no-name-e2bc083a-612e-4c3b-b70a-4c3d3dc02efe


[LightGBM] [Info] Number of positive: 3860, number of negative: 10959
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001298 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1287
[LightGBM] [Info] Number of data points in the train set: 14819, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260476 -> initscore=-1.043494
[LightGBM] [Info] Start training from score -1.043494
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[I 2025-07-02 13:43:05,532] Trial 0 finished with value: 0.9690672519477795 and parameters: {'n_estimators': 703, 'learning_rate': 0.10183482584463473, 'max_depth': 6, 'num_leaves': 80, 'subsample': 0.6286135418109621, 'colsample_bytree': 0.6540010077841952, 'reg_alpha': 2.2838689396091274, 'reg_lambda': 2.1432016772912483}. Best is trial 0 with value: 0.9690672519477795.


[LightGBM] [Info] Number of positive: 3860, number of negative: 10960
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001306 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1287
[LightGBM] [Info] Number of data points in the train set: 14820, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260459 -> initscore=-1.043585
[LightGBM] [Info] Start training from score -1.043585
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2025-07-02 13:43:06,911] Trial 1 finished with value: 0.9691751995872719 and parameters: {'n_estimators': 270, 'learning_rate': 0.0444248346153754, 'max_depth': 6, 'num_leaves': 70, 'subsample': 0.6010849932114016, 'colsample_bytree': 0.6154368434073646, 'reg_alpha': 2.7665313998173735, 'reg_lambda': 3.302995817934411}. Best is trial 1 with value: 0.9691751995872719.


...

[LightGBM] [Info] Number of positive: 3860, number of negative: 10959
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1287
[LightGBM] [Info] Number of data points in the train set: 14819, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260476 -> initscore=-1.043494
[LightGBM] [Info] Start training from score -1.043494
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2025-07-02 13:43:25,619] Trial 28 finished with value: 0.9690672519477793 and parameters: {'n_estimators': 303, 'learning_rate': 0.05864977071625574, 'max_depth': 5, 'num_leaves': 99, 'subsample': 0.7727082538670845, 'colsample_bytree': 0.9458198052170034, 'reg_alpha': 2.9403918080435125, 'reg_lambda': 4.282483854168188}. Best is trial 22 with value: 0.9693371720545757.


[LightGBM] [Info] Number of positive: 3860, number of negative: 10959
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001286 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1287
[LightGBM] [Info] Number of data points in the train set: 14819, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260476 -> initscore=-1.043494
[LightGBM] [Info] Start training from score -1.043494
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2025-07-02 13:43:26,381] Trial 29 finished with value: 0.9691752141610047 and parameters: {'n_estimators': 761, 'learning_rate': 0.10103242828882125, 'max_depth': 7, 'num_leaves': 76, 'subsample': 0.6648155003117376, 'colsample_bytree': 0.772007691876227, 'reg_alpha': 3.498504560079062, 'reg_lambda': 2.2363099303744374}. Best is trial 22 with value: 0.9693371720545757.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [13]:
# Jeśli nie masz catboost lub optuna, odkomentuj poniższą linię:
# %pip install catboost optuna

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import optuna
from catboost import CatBoostClassifier

# Jeśli masz zmienne kategoryczne, podaj ich indeksy lub nazwy:
# cat_features_cat = [lista_indeksów_lub_nazw_kolumn]
cat_features_cat = []  # <- jeśli nie masz, zostaw pustą listę

# 1. Optuna - optymalizacja hiperparametrów
def objective_cat(trial):
    param_cat = {
        'iterations': trial.suggest_int('iterations', 200, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_strength': trial.suggest_float('random_strength', 0.1, 2.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_seed': 42,
        'verbose': 0,
        'loss_function': 'Logloss',
        'eval_metric': 'Accuracy',
        'cat_features': cat_features_cat
    }
    skf_cat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores_cat = []
    for train_idx, val_idx in skf_cat.split(X_train, y_train):
        X_tr_cat, X_val_cat = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr_cat, y_val_cat = y_train.iloc[train_idx], y_train.iloc[val_idx]
        model_cat = CatBoostClassifier(**param_cat)
        model_cat.fit(X_tr_cat, y_tr_cat, eval_set=(X_val_cat, y_val_cat), early_stopping_rounds=50, use_best_model=True)
        val_pred_cat = model_cat.predict(X_val_cat)
        score_cat = accuracy_score(y_val_cat, val_pred_cat)
        scores_cat.append(score_cat)
    return np.mean(scores_cat)

study_cat = optuna.create_study(direction='maximize')
study_cat.optimize(objective_cat, n_trials=30)  # Możesz zwiększyć n_trials dla lepszych wyników

print("Najlepsze parametry:", study_cat.best_params)

# 2. Finalny model z OOF predictions i predykcjami na testach
best_params_cat = study_cat.best_params
best_params_cat.update({'random_seed': 42, 'verbose': 0, 'loss_function': 'Logloss', 'eval_metric': 'Accuracy', 'cat_features': cat_features_cat})

skf_cat = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores_cat = []
test_preds_cat = np.zeros((len(X_test), skf_cat.n_splits))
oof_preds_cat = np.zeros(len(X_train))

for fold, (train_idx, val_idx) in enumerate(skf_cat.split(X_train, y_train)):
    X_tr_cat, X_val_cat = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr_cat, y_val_cat = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    model_cat = CatBoostClassifier(**best_params_cat)
    model_cat.fit(X_tr_cat, y_tr_cat, eval_set=(X_val_cat, y_val_cat), early_stopping_rounds=50, use_best_model=True)
    
    val_pred_cat = model_cat.predict(X_val_cat)
    score_cat = accuracy_score(y_val_cat, val_pred_cat)
    scores_cat.append(score_cat)
    
    # OOF predictions
    oof_preds_cat[val_idx] = model_cat.predict_proba(X_val_cat)[:, 1]
    # Test predictions
    test_preds_cat[:, fold] = model_cat.predict_proba(X_test)[:, 1]

# OOF accuracy
oof_binary_cat = (oof_preds_cat > 0.5).astype(int)
print("OOF accuracy:", accuracy_score(y_train, oof_binary_cat))

# Uśrednienie predykcji po foldach
mean_preds_cat = test_preds_cat.mean(axis=1)
final_test_pred_cat = (mean_preds_cat > 0.5).astype(int)
print(f'Fold accuracy: {scores_cat}')
print(f'Mean CV accuracy: {np.mean(scores_cat):.4f}')

[I 2025-07-02 13:43:41,258] A new study created in memory with name: no-name-971f4513-9f87-4803-b3f5-4a0a850b1851
[I 2025-07-02 13:43:49,158] Trial 0 finished with value: 0.9694451634152669 and parameters: {'iterations': 247, 'learning_rate': 0.1605756451064026, 'depth': 10, 'l2_leaf_reg': 3.8481888766505934, 'random_strength': 0.9606699731184738, 'bagging_temperature': 0.25777813001248684, 'border_count': 75}. Best is trial 0 with value: 0.9694451634152669.
[I 2025-07-02 13:43:51,463] Trial 1 finished with value: 0.9693911677349213 and parameters: {'iterations': 1139, 'learning_rate': 0.12670445411595052, 'depth': 5, 'l2_leaf_reg': 4.615349805214294, 'random_strength': 0.3311465085568036, 'bagging_temperature': 0.8605488644931247, 'border_count': 32}. Best is trial 0 with value: 0.9694451634152669.
[I 2025-07-02 13:43:53,583] Trial 2 finished with value: 0.9691212330543921 and parameters: {'iterations': 356, 'learning_rate': 0.02233205785944426, 'depth': 5, 'l2_leaf_reg': 5.4779936855

Najlepsze parametry: {'iterations': 804, 'learning_rate': 0.24450699913516766, 'depth': 10, 'l2_leaf_reg': 9.411552757973391, 'random_strength': 0.34101041449786185, 'bagging_temperature': 0.23371379172763707, 'border_count': 53}
OOF accuracy: 0.9694990282876269
Fold accuracy: [0.967620075553157, 0.9719373988127361, 0.9730167296276309, 0.9638424177010254, 0.9654427645788337, 0.9676025917926566, 0.9719222462203023, 0.9703023758099352, 0.9713822894168467, 0.9719222462203023]
Mean CV accuracy: 0.9695


In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import optuna
import xgboost as xgb

# 1. Optuna - optymalizacja hiperparametrów
def objective_xgb(trial):
    param_xgb = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
        'random_state': 42,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss',
        'early_stopping_rounds': 50
    }
    skf_xgb = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores_xgb = []
    for train_idx, val_idx in skf_xgb.split(X_train, y_train):
        X_tr_xgb, X_val_xgb = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr_xgb, y_val_xgb = y_train.iloc[train_idx], y_train.iloc[val_idx]
        model_xgb = xgb.XGBClassifier(**param_xgb)
        model_xgb.fit(X_tr_xgb, y_tr_xgb,
                      eval_set=[(X_val_xgb, y_val_xgb)],
                      verbose=False)
        scores_xgb.append(accuracy_score(y_val_xgb, model_xgb.predict(X_val_xgb)))
    return np.mean(scores_xgb)

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=30)
print("Najlepsze parametry:", study_xgb.best_params)

best_params_xgb = study_xgb.best_params
best_params_xgb.update({
    'random_state': 42,
    'n_jobs': -1,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'early_stopping_rounds': 50
})

skf_xgb = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores_xgb = []
test_preds_xgb = np.zeros((len(X_test), skf_xgb.n_splits))
oof_preds_xgb = np.zeros(len(X_train))

for fold, (train_idx, val_idx) in enumerate(skf_xgb.split(X_train, y_train)):
    X_tr_xgb, X_val_xgb = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr_xgb, y_val_xgb = y_train.iloc[train_idx], y_train.iloc[val_idx]
    model_xgb = xgb.XGBClassifier(**best_params_xgb)
    model_xgb.fit(X_tr_xgb, y_tr_xgb,
                  eval_set=[(X_val_xgb, y_val_xgb)],
                  verbose=False)
    scores_xgb.append(accuracy_score(y_val_xgb, model_xgb.predict(X_val_xgb)))
    oof_preds_xgb[val_idx] = model_xgb.predict_proba(X_val_xgb)[:, 1]
    test_preds_xgb[:, fold] = model_xgb.predict_proba(X_test)[:, 1]

oof_binary_xgb = (oof_preds_xgb > 0.5).astype(int)
print("OOF accuracy:", accuracy_score(y_train, oof_binary_xgb))
mean_score_xgb = np.mean(scores_xgb)
print("Fold accuracy:", scores_xgb)
print(f"Mean CV accuracy: {mean_score_xgb:.4f}")

mean_preds_xgb = test_preds_xgb.mean(axis=1)
final_test_pred_xgb = (mean_preds_xgb > 0.5).astype(int)

[I 2025-07-02 13:45:56,738] A new study created in memory with name: no-name-0c2fc379-1c79-4e44-b924-d4058774229a


c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\callback.py:386: UserWarning: [13:45:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()
c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\callback.py:386: UserWarning: [13:45:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()
c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\callback.py:386: UserWarning: [13:45:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()
c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\callback.py:386: UserWarning: [13:45:58] WARNING:

Najlepsze parametry: {'n_estimators': 839, 'learning_rate': 0.24816900099301387, 'max_depth': 8, 'subsample': 0.6838681298252383, 'colsample_bytree': 0.6502983264483474, 'gamma': 4.992770692624027, 'reg_alpha': 1.168069559577995, 'reg_lambda': 2.7862920179654886}


c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\callback.py:386: UserWarning: [13:46:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()
c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\callback.py:386: UserWarning: [13:46:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()
c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\callback.py:386: UserWarning: [13:46:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()
c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\callback.py:386: UserWarning: [13:46:21] WARNING:

OOF accuracy: 0.969067156121788
Fold accuracy: [0.9681597409606044, 0.9719373988127361, 0.9724770642201835, 0.963302752293578, 0.964902807775378, 0.9659827213822895, 0.9713822894168467, 0.9708423326133909, 0.9703023758099352, 0.9713822894168467]
Mean CV accuracy: 0.9691


In [15]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# OOF predictions ensemble (średnia po modelach)
oof_preds_ensemble = (oof_preds_lgbm + oof_preds_cat + oof_preds_xgb) / 3
oof_binary_ensemble = (oof_preds_ensemble > 0.5).astype(int)
oof_accuracy_ensemble = accuracy_score(y_train, oof_binary_ensemble)
print("OOF accuracy (ensemble):", oof_accuracy_ensemble)

# CV accuracy ensemble (accuracy na każdym foldzie)
skf_ensemble = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores_ensemble = []

for fold, (train_idx, val_idx) in enumerate(skf_ensemble.split(X_train, y_train)):
    oof_fold = (oof_preds_lgbm[val_idx] + oof_preds_cat[val_idx] + oof_preds_xgb[val_idx]) / 3
    oof_fold_binary = (oof_fold > 0.5).astype(int)
    acc = accuracy_score(np.array(y_train)[val_idx], oof_fold_binary)
    cv_scores_ensemble.append(acc)

mean_cv_accuracy_ensemble = np.mean(cv_scores_ensemble)
print(f"Fold accuracy (ensemble): {cv_scores_ensemble}")
print(f"Mean CV accuracy (ensemble): {mean_cv_accuracy_ensemble:.4f}")

# Test predictions ensemble (średnia po modelach i foldach)
mean_preds_lgbm = test_preds_lgbm.mean(axis=1)
mean_preds_cat = test_preds_cat.mean(axis=1)
mean_preds_xgb = test_preds_xgb.mean(axis=1)
ensemble_test = (mean_preds_lgbm + mean_preds_cat + mean_preds_xgb) / 3
final_ensemble_pred = (ensemble_test > 0.5).astype(int)

OOF accuracy (ensemble): 0.9691751241632477
Fold accuracy (ensemble): [0.967620075553157, 0.9719373988127361, 0.9724770642201835, 0.963302752293578, 0.9654427645788337, 0.9670626349892009, 0.9713822894168467, 0.9708423326133909, 0.9703023758099352, 0.9713822894168467]
Mean CV accuracy (ensemble): 0.9692


In [16]:
import sys
sys.path.append('../src')

from experiment_logger import log_experiment

params = {
    'models': ['LGBMClassifier', 'CatBoostClassifier', 'XGBClassifier'],
    'ensemble_type': 'mean_probability',
    'n_splits': 10
}

log_experiment(
    experiment_name='ensemble_lgbm_cat_xgb_mean',
    model_name='EnsembleMean',
    params=params,
    cv_score=mean_cv_accuracy_ensemble,  # mean CV accuracy ensemble
    comment='Ensemble średnia prawdopodobieństw z LGBM, CatBoost, XGBoost; 10-fold CV; OOF accuracy logowane; imputacja numeryczna i kategoryczna (jeśli była)',
    oof_accuracy=oof_accuracy_ensemble   # OOF accuracy ensemble
)
print('Eksperyment ensemble został zalogowany!')

Eksperyment ensemble został zalogowany!


In [ ]:
import os
# Wczytaj sample_submission, aby pobrać wymagane kolumny i kolejność
sample_submission = pd.read_csv('../../playground-series-s5e7/sample_submission.csv')

# Zakładam, że predykcje ensemble są w zmiennej final_ensemble_pred (np. jako liczby lub kategorie)
if set(np.unique(final_ensemble_pred)) == {0, 1}:
    label_map = {0: 'Extrovert', 1: 'Introvert'}
    final_ensemble_pred_labels = pd.Series(final_ensemble_pred).map(label_map).values
else:
    final_ensemble_pred_labels = final_ensemble_pred

submission = sample_submission.copy()
target_col = submission.columns[1]
submission[target_col] = final_ensemble_pred_labels

# Automatyczne nadawanie nazwy pliku
output_dir = '../outputs'
existing = [f for f in os.listdir(output_dir) if f.startswith('submission') and f.endswith('.csv')]
if 'submission.csv' in existing:
    # Szukamy submissionN.csv
    nums = [int(f.replace('submission', '').replace('.csv', '')) for f in existing if f != 'submission.csv' and f.replace('submission', '').replace('.csv', '').isdigit()]
    n = max(nums) if nums else 1
    new_name = f'submission{n+1}.csv'
else:
    new_name = 'submission.csv'

output_path = os.path.join(output_dir, new_name)
submission.to_csv(output_path, index=False)
print(f'Plik submission zapisany do {output_path}')
submission.head()

Plik submission zapisany do ../outputs\submission8.csv


,id,Personality
0,18524,Extrovert
1,18525,Introvert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Introvert
